In [1]:
import os
import math

In [2]:
'''
@breif: function for debugging
@param x: input
@return y
'''

def fun3(x):
    y = (1.0/3.0)*x**3 - 0.5*x**2 - x - 1.0
    return y

def func(x):
    y = math.cos(x)
    return y

'''
@breif: parabolic interpolation method
@param x1, y1, x2, y2, x3, y3: three points
@return p, q: x4 = x3 + p/q               
'''

# x3<x2, x1
#x = (x1^2*(y2-y3)+x2^2*(y3-y1)+x3^2*(y1-y2))/(2*(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2)))
#which is invariant under the permutation of the three points
def parabolic_interpolation(x1, y1, x2, y2, x3, y3):
    p = (x2 - x3)**2 * (y3 - y1) + (x1 - x3)**2 * (y2 - y3)
    q = 2.0 * ((x2 - x3) * (y3 - y1) + (x1 - x3) * (y2 - y3))
    return p, q
    
'''
@breif: brent method in one iteration, in one iteration, only one new point is evaluated(fu)

@param a, b: boundary of the interval
@param v, fv: the old value of w
@param w, fw: second least value
@param x, fx: the least value
@param fun_p: the function to be minimized
@param dx: the last step length
@param dxold: the last last step length

@return fv, fw, fx: updated values
@return a, b, v, w, x, dx, dxold: updated values
@return u: last evaluated point
'''

#* here I updated the values of fv, fw, fx to save the computation time, only fu is evaluated in one iteration
def brent(a, b, v, fv, w, fw, x, fx, fun_p, dx, dxold):
    CGOLD = 0.381966013
    xm = 0.5 * (a + b)

    #perform parabolic interpolation using v, w, x (x the minimum point)
    p, q = parabolic_interpolation(v, fv, w, fw, x, fx)

    #Boolean variables for the safety of parabolic interpolation
    #* qSafe: q is not zero
    qSafe = True if (q != 0) else False

    tempdx = p/q if qSafe else 0.0
    #* paraSafe:
    #* 1. q is not zero
    #* 2. the new point is within the interval
    #* 3. the new step length is less than half of the last last step length
    paraSafe = True if (qSafe and (a < x+tempdx) and (x+tempdx < b) and (abs(tempdx) < 0.5*abs(dxold))) else False

    #update the dx and store the last last step length for next iteration

    if paraSafe:
        dxold = dx
        dx = tempdx
    else:
        if x < xm:
            dxold = b - x
        else:
            dxold = a - x
        dx = CGOLD * dxold

    #! evaluate the new point
    u = x + dx
    fu = fun_p(u)

    if (fu < fx):
        #* update the boundary of interval
        a = a if (u < x) else x
        b = x if (u < x) else b
        #* update the values of v, w, x
        v = w
        fv = fw
        w = x
        fw = fx
        x = u
        fx = fu
    else:
        #* update the boundary of interval
        a = u if (u < x) else a
        b = b if (u < x) else u

        #* update the values of v, w, x
        if (fu < fw) or (w == x):
            v = w
            fv = fw
            w = u
            fw = fu
        elif (fu < fv) or (v == x) or (v == w):
            v = u
            fv = fu
    
    return a, b, v, fv, w, fw, x, fx, dx, dxold

'''
@breif: brent method
@param a0, b0: initial boundary of the interval
@param xguess: initial guess of the minimum point
@param fun_p: the function to be minimized
@param tol: tolerance

@return xmin: the minimum point
@return icount: the number of iterations
'''
def brentMethod(a0, b0, xguess, fun_p, tol):
    Maxloop = 100

    icount = 0
    #! evaluate the function at the initial guess
    y0 = fun_p(xguess)
    a, b, v, fv, w, fw, x, fx, dx, dxold = brent(a0, b0, xguess, y0, xguess, y0, xguess, y0, fun_p, 0.0, 0.0)

    print(f"x:\t{x:.6f}\ty:\t{fx:.12f}")

    for i in range(Maxloop):
        a, b, v, fv, w, fw, x, fx, dx, dxold = brent(a, b, v, fv, w, fw, x, fx, fun_p, dx, dxold)

        print(f"x:\t{x:.6f}\ty:\t{fx:.12f}")

        icount += 1
        if (abs(b - a) < tol):
            xmin = 0.5 * (a + b)
            break
            
    return xmin, icount
        


In [3]:
xmin, icount = brentMethod(0.0, 2.0, 6.0, func, 1.0e-5)
print(f"Minimum point: {xmin} Iteration: {icount} ")

x:	3.708204	y:	-0.843724805852
x:	3.708204	y:	-0.843724805852
x:	3.262151	y:	-0.992741587697
x:	3.135621	y:	-0.999982168936
x:	3.139712	y:	-0.999998231072
x:	3.141597	y:	-0.999999999990
x:	3.141593	y:	-1.000000000000
x:	3.141593	y:	-1.000000000000
Minimum point: 3.141594879802334 Iteration: 7 


In [4]:
xmin, icount = brentMethod(0.0, 2.0, 4.0, fun3, 1.0e-5)
print(f"Minimum point: {xmin} Iteration: {icount} ")

x:	2.472136	y:	-1.491747210146
x:	1.527864	y:	-2.506182393311
x:	1.527864	y:	-2.506182393311
x:	1.621020	y:	-2.515018348785
x:	1.616276	y:	-2.515024871313
x:	1.618016	y:	-2.515028323606
x:	1.618033	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
x:	1.618034	y:	-2.515028323958
Minimum point: 1.6180386286234483 Iteration: 15 


In [5]:
'''
@breif: find the spin index in the gjf file
by locating the second blank line index, then add 1

@param filename: default filename is "template.gjf"

@retrun iopIndex: the index of the iop line (start from 1)
@return spinIndex: the index of the spin line (start from 1)
'''
def findSpinIndex(filename = "template.gjf"):

    with open(filename, "r") as gjf:
        icount = 0
        blankCount = 0
        
        for line in gjf:
            icount += 1
            if line == "\n":
                blankCount += 1
            if blankCount == 2:
                spinIndex = icount + 1
                break
        
        iopIndex = spinIndex - 4

    return iopIndex, spinIndex

'''
@brief: generate gjf with N+1, N, N-1 spins in different IOp(3/107, 3/108) values (int(wpara*1000):05 + 0:05 format)

@param Nspin: "charge spin" of N
@param Naspin: "charge spin" of N+1
@param Nmspin: "charge spin" of N-1
@param wpara: the parameter for the IOp(3/107, 3/108) value
@param filename: the template file name, default is "template.gjf"


@return: void, three gjf files: N.gjf, N+1.gjf, N-1.gjf
'''
#! here I use f"{int(wpara*10000):05}" to convert the float to string, by adding 5 zeros

def g16input(Nspin, Naspin, Nmspin, wpara, filename = "template.gjf"):

    iopIndex, spinIndex = findSpinIndex()

    with open("N.gjf", "w") as Ngjf, open("N+1.gjf", "w") as Nagjf, open("N-1.gjf", "w") as Nmgjf:
        with open(filename, "r") as gjf:
            icount = 0
            for line in gjf:
                icount += 1
                if icount == iopIndex:
                    tempIop = line.strip()  #*remove the newline character
                    str_wpara = f"{int(wpara*10000):05}"   #*convert the float to string, by adding 5 zeros
                    str_iop = str_wpara + f"{0:05}"
                    
                    Ngjf.write(f"{tempIop} IOp(3/107={str_iop},3/108={str_iop})\n")
                    Nagjf.write(f"{tempIop} IOp(3/107={str_iop},3/108={str_iop})\n")
                    Nmgjf.write(f"{tempIop} IOp(3/107={str_iop},3/108={str_iop})\n")

                elif icount == spinIndex:
                    Ngjf.write(f"{Nspin}\n")
                    Nagjf.write(f"{Naspin}\n")
                    Nmgjf.write(f"{Nmspin}\n")
                else:
                    Ngjf.write(line)
                    Nagjf.write(line)
                    Nmgjf.write(line)


'''
@brief: read the SCF energy and eigenvalues of HOMO orbital

@param: filename the log file

@return: SCFenergy, eHOMO
'''

#* NOTICE: sometimes there will be not one SCF iterations in a log file,
#* such as stable=opt, and we take the newest values
def g16read(filename):
    success = False

    with open(filename, "r") as log:
        for line in log:
            if "SCF Done" in line:
                templine = line.split()
                SCFenergy = float(templine[4])
            if "Alpha virt. eigenvalues" in line and "Alpha  occ. eigenvalues" in prev_line:
                templine = prev_line.split()
                eHOMO = float(templine[-1])
            #* check if succeed
            if "Normal termination" in line:
                success = True
            prev_line = line    #! prev_line should be updated in the end of line

        if not success:
            raise Exception("Error: Gaussian doesn't finish correctly. (Does not finished normally.)")
    
    return SCFenergy, eHOMO

In [7]:
#! only in my computer

'''
@brief: read the energy from the Gaussian output file, as a function y(x)

@param wpara: parameter of functional LC-wPBE, as x
@return: J2, as y

@global: Nspin, Naspin, Nmspin
'''

def funJ2(wpara):
    global Nspin, Naspin, Nmspin

    g16input(Nspin, Naspin, Nmspin, wpara)
    #* run g16
    #! remove the module load command later
    exitcode1 = os.system("module load Gaussian16-B01;g16 < N.gjf > N.log")
    exitcode2 = os.system("module load Gaussian16-B01;g16 < N+1.gjf > N+1.log")
    exitcode3 = os.system("module load Gaussian16-B01;g16 < N-1.gjf > N-1.log")

    #* check the exit code (non-zero means error)
    if exitcode1 or exitcode2 or exitcode3:
        print("Error in Gaussian16")
        raise Exception("Error: Gaussian doesn't finish correctly. (exitcode does not equal to 0)")

    #* store the output files
    tempstr = f"{int(wpara*10000):05}" 
    os.system(f"mkdir {tempstr}")
    os.system(f"cp *.log {tempstr}")

    #* extract the energy
    EN, eN = g16read("N.log")
    ENa, eNa = g16read("N+1.log")
    ENm, eNm = g16read("N-1.log")

    JN = abs(eN + ENm - EN)
    JNa = abs(eNa + EN - ENa)
    J2 = JN**2 + JNa**2

    return J2



In [9]:
J2 = funJ2(0.4387)
print(J2)

Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi


5.657249742848371e-05


mkdir: cannot create directory ‘04387’: File exists


In [9]:
J2 = funJ2(0.439055)
print(J2)

Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi


5.649663248766607e-05


mkdir: cannot create directory ‘04390’: File exists


In [8]:
Nspin = "0 1"
Naspin = "-1 2"
Nmspin = "1 2"

xmin, icount = brentMethod(0.05, 0.325, 0.6, funJ2, 1.0e-3)

Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
mkdir: cannot create directory ‘06000’: File exists
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
mkdir: cannot create directory ‘03899’: File exists


x:	0.389919	y:	0.000179826572


Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
mkdir: cannot create directory ‘02600’: File exists


x:	0.389919	y:	0.000179826572


Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
mkdir: cannot create directory ‘04688’: File exists


...

x:	0.439055	y:	0.000056496632


Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
mkdir: cannot create directory ‘04380’: File exists


x:	0.439055	y:	0.000056496632


Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
Loading Gaussian16-B01
  Loading requirement: oneapi
mkdir: cannot create directory ‘04387’: File exists


x:	0.439055	y:	0.000056496632
